## Importing all the libraries

In [1]:
import numpy as np
import random
import re
from nltk.util import ngrams
import itertools
import pandas as pd
from sklearn import svm
from joblib import dump, load
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

## Creating all the possible combinations of n-grams using lowercase alphabets and digits and storing in a dictionary

In [2]:
#np.set_printoptions(threshold=np.inf)
alphanum = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','0','1','2','3','4','5','6','7','8','9']
permutations = itertools.product(alphanum, repeat=3)
featuresDict = {}
counter = 0
for perm in permutations:
    #print(perm)
    f=''
    for char in perm:
        f = f+char;
    featuresDict[(''.join(perm))] = counter
    counter = counter + 1

## A function that takes a sentence as imput and returns list of n-grams

In [3]:
def generate_ngram(sentence):
    s = sentence.lower()
    s = ''.join(e for e in s if e.isalnum()) #replace spaces and slashes
    processedList = []
    for tup in list(ngrams(s,3)):
        processedList.append((''.join(tup)))
    return processedList

## A function that accepts a dataframe and 2 arrays X & y. X denotes the features that will be used for training and y is the label 
## URL is then stripped down to the suitable format for pre-processing like removing unwanted Dots, Slash , WWW. , .com etc.
## This data is then transferred to the pre-processing module for conversion into suitable format for our model.

In [4]:
def preprocess_sentences(dataframe, X, y):
    #print(dataframe)
    for index,row in df.iterrows():
        url = row['url'].strip().replace("https://","")
        url = row['url'].strip().replace("http://","")
        url = url.replace("http://","")
        url = re.sub(r'\.[A-Za-z0-9]+/*','',url)
        for gram in generate_ngram(url):
            try:
                X[index][featuresDict[gram]] = X[index][featuresDict[gram]] + 1
            except:
                print(gram,"doesn't exist")
        y[index] = int(row['label'])
    return (X,y)

## Load the dataset and split it into train and test set

In [5]:
url_dataframe = pd.read_csv("malicious-url-detector_dataset.csv")
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

## The dataset has 1 million rows. 
## we will be using SGDClassifier which by default fits a linear support vector machine (SVM).
## This estimator implements regularized linear models with stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate). SGD allows minibatch (online/out-of-core) learning via the partial_fit method.

In [6]:
no_of_rows = 25000
no_of_batches = int(train_df.shape[0]/no_of_rows) +1
classifier = SGDClassifier()
#print(no_of_batches)
for i in range(0, no_of_batches):
    start = no_of_rows*i
    if start + no_of_rows > train_df.shape[0] :
        df = train_df.iloc[start:,:]
    else :
        df = train_df.iloc[start:start+no_of_rows, :]
    df = df.reset_index()
    (X,y) = preprocess_sentences(df, \
                                 np.zeros([df.shape[0], 46656],dtype="int"), \
                                 np.zeros(df.shape[0],dtype="int"))
    classifier.partial_fit(X, y, classes=np.unique(y))

aiه doesn't exist
iه½ doesn't exist
ه½è doesn't exist
½è¹ doesn't exist
è¹ه doesn't exist
¹هن doesn't exist
هن½ doesn't exist
ن½ه doesn't exist
½ه½ doesn't exist
ه½è doesn't exist
½è¹ doesn't exist
è¹ه doesn't exist
¹هڈ doesn't exist
هڈو doesn't exist
ڈوه doesn't exist
وهç doesn't exist
هç² doesn't exist
ç²ن doesn't exist
²نه doesn't exist
نهé doesn't exist
هéن doesn't exist
éن½ doesn't exist
18æ doesn't exist
8æ³ doesn't exist
æ³å doesn't exist
³åœ doesn't exist
åœæ doesn't exist
œæœ doesn't exist
æœº doesn't exist
œº3 doesn't exist
º32 doesn't exist
e4ä doesn't exist
4ää doesn't exist
ääš doesn't exist
äšç doesn't exist
šçè doesn't exist
çèç doesn't exist
èçˆ doesn't exist
çˆï doesn't exist
ˆï¼ doesn't exist
ï¼ˆ doesn't exist
¼ˆå doesn't exist
ˆåå doesn't exist
ååº doesn't exist
åºå doesn't exist
ºåˆ doesn't exist
åˆå doesn't exist
ˆåï doesn't exist
åï¼ doesn't exist
ï¼3 doesn't exist
¼32 doesn't exist
wnا doesn't exist
nاه doesn't exist
اهء doesn't exist
هءم doesn't exist
aiه doesn'

## Testing the model against the test set and getting the count of correct and incorrect predictions

In [ ]:
no_of_rows = 25000
no_of_batches = int(test_df.shape[0]/no_of_rows) +1
#print(no_of_batches)
correct = 0;
incorrect = 0
for i in range(0, no_of_batches):
    start = no_of_rows*i
    if start + no_of_rows > train_df.shape[0] :
        df = train_df.iloc[start:,:]
    else :
        df = test_df.iloc[start:start+no_of_rows, :]
    df = df.reset_index()
    (X_test,y_test) = preprocess_sentences(df, \
                                 np.zeros([df.shape[0], 46656],dtype="int"), \
                                 np.zeros(df.shape[0],dtype="int"))
    y_pred = classifier.predict(X_test)
    for index,row in df.iterrows():
        if row['label'] == y_pred[index]:
            correct = correct+1
        else:
            incorrect = incorrect + 1        

## Displaying the result of the model

In [ ]:
print("Correct Predictions ", correct)
print("Incorrect Predictions ", incorrect)
accuracy = (correct/test_df.shape[0])*100
print("Accuracy of the model is: "'{0:.4g}'.format(accuracy), '%')

In [8]:
def preprocess_sentences_url(url):
    X= np.zeros([1, 46656],dtype="int")
    url = url.strip().replace("https://","")
    url = url.replace("http://","")
    url = re.sub(r'\.[A-Za-z0-9]+/*','',url)
    for gram in generate_ngram(url):
        try:
            X[0][featuresDict[gram]] = X[0][featuresDict[gram]] + 1
            #print('preprocess_sentences')
            #print(X[index][featuresDict[gram]])
        except:
            print(gram,"doesn't exist")
    return X        

In [17]:
url = "www.itidea.it/centroesteticosothys/img/_notes/gum.exe"
test = preprocess_sentences_url(url)
pred = classifier.predict(test)
if pred == 1 :
    print(url, " is malacious!")
else:
    print(url, " is not malacious!")

www.itidea.it/centroesteticosothys/img/_notes/gum.exe  is malacious!
